In [1]:
from gensim import models, corpora, similarities
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from nltk.tokenize import TweetTokenizer
import pyemd
import csv
import multiprocessing

cores = multiprocessing.cpu_count()
print cores

8


In [2]:
with open('questions.csv', 'rU') as f:
    f.next()
    reader = csv.reader(f)
    qpairs = list(reader)
   
tknzr = TweetTokenizer()
qcorp = []
qdict = {}
for qpair in qpairs:
    qid = qpair[1]
    qwords = tknzr.tokenize(qpair[3])
    q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
    qcorp.append(q)
    qdict["QID_"+str(qid)] = qwords
    qid = qpair[2]
    qwords = tknzr.tokenize(qpair[4])
    q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
    qcorp.append(q)
    qdict["QID_"+str(qid)] = qwords

In [182]:
model = Doc2Vec(size=100, window=5, negative=5, sample=1e-4, min_count=2, workers=cores, alpha=0.025, min_alpha=0.025)
model.build_vocab(qcorp)
for epoch in range(10):
    model.train(qcorp)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    model.train(qcorp)

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10


In [183]:
model.save("doc2vec.model")

In [4]:
model = Doc2Vec.load("doc2vec.model")

In [5]:
vectors = model.docvecs

In [6]:
for i in range(len(qpairs)):
    qid1 = "QID_"+str(qpairs[i][1])
    qid2 = "QID_"+str(qpairs[i][2])
    score = vectors.similarity(qid1, qid2)
    qpairs[i].extend([score])

In [9]:
for i in range(25):
    print qpairs[i]

['0', '1', '2', 'What is the step by step guide to invest in share market in india?', 'What is the step by step guide to invest in share market?', '0', 0.82047855664681113]
['1', '3', '4', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?', '0', 0.45048160764946837]
['2', '5', '6', 'How can I increase the speed of my internet connection while using a VPN?', 'How can Internet speed be increased by hacking through DNS?', '0', 0.34205746999516351]
['3', '7', '8', 'Why am I mentally very lonely? How can I solve it?', 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?', '0', 0.25290592385833743]
['4', '9', '10', 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?', 'Which fish would survive in salt water?', '0', 0.59393453045042799]
['5', '11', '12', 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', "I'm a triple Cap

In [187]:
wordvectors = model.wv

In [193]:
print wordvectors.most_similar(["UPSC"])

[(u'IAS', 0.8955783247947693), (u'CAT', 0.843267560005188), (u'GATE', 0.8333038091659546), (u'CSE', 0.8325021266937256), (u'exams', 0.8290932178497314), (u'IIT', 0.8272045850753784), (u'SSC', 0.8266948461532593), (u'exam', 0.8232311010360718), (u'graduation', 0.8177750110626221), (u'college', 0.814316987991333)]


In [189]:
test = "QID_15"
tid = int(test[4:]) - 1
print qcorp[tid]
for q in vectors.most_similar(["QID_15"]):
    print q, qdict[q[0]]

TaggedDocument([u'How', u'can', u'I', u'be', u'a', u'good', u'geologist', u'?'], ['QID_15'])
('QID_149134', 0.7738628387451172) [u'How', u'can', u'I', u'be', u'good', u'programer', u'?']
('QID_627259', 0.7621285319328308) [u'How', u'can', u'I', u'be', u'a', u'good', u'software', u'engineer', u'?']
('QID_16', 0.7554548978805542) [u'What', u'should', u'I', u'do', u'to', u'be', u'a', u'great', u'geologist', u'?']
('QID_604540', 0.7509888410568237) [u'How', u'can', u'I', u'be', u'good', u'engineer', u'?']
('QID_71714', 0.7502573728561401) [u'How', u'can', u'I', u'be', u'a', u'good', u'chess', u'player', u'?']
('QID_410466', 0.7500404119491577) [u'How', u'can', u'I', u'be', u'a', u'good', u'husband', u'for', u'my', u'wife', u'?']
('QID_557277', 0.7480751872062683) [u'How', u'can', u'I', u'be', u'good', u'programer', u'?']
('QID_588867', 0.7438502907752991) [u'How', u'can', u'I', u'be', u'a', u'good', u'husband', u'for', u'my', u'wife', u'?']
('QID_384999', 0.7418942451477051) [u'How', u'can